In [1]:
import rasterio
import rasterio.features
import rasterio.warp
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot
import numpy as np
import pandas as pd
import os

In [2]:
df = pd.read_csv('data_processed.csv')

In [3]:
#x: longitud y: latitud
x = -72.6983	
y = -42.4087
df

,Región,Comuna,Tipo Remoción en masa,Factor desencadenante,Fecha Evento,Longitud,Latitud,Sistema Georeferencia,Cota (m.s.n.m)
0,Los Lagos,Castro,Flujo,Precipitaciones,07/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
1,Los Lagos,Castro,Flujo,Precipitaciones,09/03/2001,-73.7855,-42.4686,WGS 84- UTM 18 S,-
2,Los Lagos,Chaitén,Deslizamiento,Precipitaciones,03/05/2002,-72.6983,-42.4087,WGS 84- UTM 18 S,-
3,Los Lagos,Río Negro,Deslizamiento,Precipitaciones,09/08/2002,-73.7889,-40.7083,WGS 84- UTM 18 S,6
4,Los Lagos,Osorno,Deslizamiento,Precipitaciones,22/08/2007,-73.6047,-40.5750,WGS 84- UTM 18 S,126
...,...,...,...,...,...,...,...,...,...
1242,Biobío,Tomé,Deslizamiento,Precipitaciones,11/07/2006,-72.9781,-36.5932,WGS 84/ UTM 18 S,40
1243,Biobío,Talcahuano,Caída,Precipitaciones,15/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1244,Biobío,Talcahuano,Caída,Precipitaciones,16/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-
1245,Biobío,Talcahuano,Caída,Precipitaciones,17/06/1991,-73.1032,-36.7543,WGS 84/ UTM 18 S,-


In [9]:
#Contar tipos de valores obtenido en "Tipo Remoción en masa"
conteo_tipos_remocion = df["Tipo Remoción en masa"].value_counts()

print("Conteo de valores en la columna 'Tipo de remoción en masa':")
print(conteo_tipos_remocion)


Conteo de valores en la columna 'Tipo de remoción en masa':
Tipo Remoción en masa
Deslizamiento           978
Flujo                   153
Caída                    79
Deslizamiento, Flujo     13
Deslizamiento. Flujo     11
Deslizameinto             4
Caída de bloques          4
deslizamiento             2
Inundación                2
Deslizamient, Flujo       1
Name: count, dtype: int64


In [13]:
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'

path_PIRange = r'PIRange-002'
path_rosetta_mean = r'ROSETTA_MEAN'
path_rosetta_sd = r'ROSETTA_SD'
path_SoilMaps_mean = r'SoilMaps_MEAN'
#path_SoilMaps_mean_2 = r'SoilMaps_MEAN'
path_Textural_Classes = r'Textural_Classes'
all_paths = [path_PIRange, path_rosetta_mean, path_rosetta_sd, path_SoilMaps_mean, path_Textural_Classes] #path_SoilMaps_mean_2
all_paths

['PIRange-002',
 'ROSETTA_MEAN',
 'ROSETTA_SD',
 'SoilMaps_MEAN',
 'Textural_Classes']

In [99]:
def archivos_extraccion(paths):
    archivos = []
    for path in paths:
        for elemento in os.listdir(path):
            ruta_archivo = os.path.join(path, elemento)
            if os.path.isfile(ruta_archivo) and  ruta_archivo.lower().endswith('.tif'):
                archivos.append(fr'{path}/{elemento}')
    return archivos

archivos = archivos_extraccion(all_paths)
archivos[0:3]

['PIRange-002/PIRange_Bulkd.0-5cm.tif',
 'PIRange-002/PIRange_Bulkd.100-200cm.tif',
 'PIRange-002/PIRange_Bulkd.15-30cm.tif']

In [27]:
for archivo in archivos:
    print(archivo.split('/')[1])

PIRange_Bulkd.0-5cm.tif
PIRange_Bulkd.100-200cm.tif
PIRange_Bulkd.15-30cm.tif
PIRange_Bulkd.30-60cm.tif
PIRange_Bulkd.5-15cm.tif
PIRange_Bulkd.60-100cm.tif
PIRange_Clay.0-5cm.tif
PIRange_Clay.100-200cm.tif
PIRange_Clay.15-30cm.tif
PIRange_Clay.30-60cm.tif
PIRange_Clay.5-15cm.tif
PIRange_Clay.60-100cm.tif
PIRange_Sand.0-5cm.tif
PIRange_Sand.100-200cm.tif
PIRange_Sand.15-30cm.tif
PIRange_Sand.30-60cm.tif
PIRange_Sand.5-15cm.tif
PIRange_Sand.60-100cm.tif
alpha_0-5cm.tif
alpha_100-200cm.tif
alpha_15-30cm.tif
alpha_30-60cm.tif
alpha_5-15cm.tif
alpha_60-100cm.tif
AvMoist.0-5cm.tif
AvMoist.100-200cm.tif
AvMoist.15-30cm.tif
AvMoist.30-60cm.tif
AvMoist.5-15cm.tif
AvMoist.60-100cm.tif
AWC_0-5cm.tif
AWC_100-200cm.tif
AWC_15-30cm.tif
AWC_30-60cm.tif
AWC_5-15cm.tif
AWC_60-100cm.tif
FC.0-5cm.tif
FC.100-200cm.tif
FC.15-30cm.tif
FC.30-60cm.tif
FC.5-15cm.tif
FC.60-100cm.tif
ksat_0-5cm.tif
ksat_100-200cm.tif
ksat_15-30cm.tif
ksat_30-60cm.tif
ksat_5-15cm.tif
ksat_60-100cm.tif
n_0-5cm.tif
n_100-200cm.tif


In [36]:
"""with rasterio.open(path) as dataset:
    mask = dataset.dataset_mask()
    for geom, val in rasterio.features.shapes(
        mask, transform=dataset.transform):
        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)
        
    print(geom)"""

"with rasterio.open(path) as dataset:\n    mask = dataset.dataset_mask()\n    for geom, val in rasterio.features.shapes(\n        mask, transform=dataset.transform):\n        geom = rasterio.warp.transform_geom(dataset.crs, 'EPSG:4326', geom, precision=6)\n        \n    print(geom)"

In [63]:
#funcions de lectura de tif
#x_,y_ = dataset_.index(-71.838400,-39.554800)
path = r'ROSETTA_MEAN/theta_r_15-30cm.tif'
dataset_ = rasterio.open(path)
x_,y_ = dataset_.index(x,y)
#print(x,y)
#print(x_,y_)
#x: longitud y: latitud
banda=dataset_.read(1)

print(banda[x_][y_])
valor = banda[x_][y_]
#print('Valor:', type(valor))

0.087


In [46]:
df = pd.DataFrame(new_df)
#df['Nombre'] = None
#nuevo = range(0,1247)
#df['Nombre'] = nuevo
df

,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543


In [47]:
nuevo = range(0,1247)
df_val = pd.DataFrame({'Nombre': nuevo})
pd.concat([df, df_val], axis=1)

,Longitud,Latitud,Nombre
0,-73.7855,-42.4686,0
1,-73.7855,-42.4686,1
2,-72.6983,-42.4087,2
3,-73.7889,-40.7083,3
4,-73.6047,-40.5750,4
...,...,...,...
1242,-72.9781,-36.5932,1242
1243,-73.1032,-36.7543,1243
1244,-73.1032,-36.7543,1244
1245,-73.1032,-36.7543,1245


In [80]:
#print(new_df.Longitud.iloc[0])
#print(new_df.iloc[0].Longitud)

-73.7855
-73.7855


In [ ]:
#x: longitud y: latitud
new_df = pd.concat([df.Longitud, df.Latitud], axis = 1)
#archivos = [path]

In [85]:
def tif_extraccion(archivos: list, latlon: pd.DataFrame):
    # archivos: [path1.tif, path2.tif, ...]
    # latlon: [Longitud , Latitud]
    #           x1          y1
    #           x2          y2
    #           ...         ...
    for archivo in archivos:
        df = pd.DataFrame(latlon)
        dataset = rasterio.open(archivo)
        nombre = archivo.split('/')[1]
        valores = []
        for i in range(len(latlon)):
            x = latlon.iloc[i].Longitud
            y = latlon.Latitud.iloc[i]
            x_, y_ = dataset.index(x,y)
            banda = dataset.read(1)
            valor= np.nan
            try:
                valor = banda[x_,y_]
            except Exception as e:
                print(f'Error {e}')
            valores.append(valor)
        df[nombre] = valores
    # return:
    # df: [Longitud, Latitud, tif1_data, tif2_data, ..., tifn_data]
    #       Long1      Lat1    data1_1    data1_2   ...   data1_n
    #       Long2      Lat2    data2_1    data2_2   ...   data2_n
    #       ...        ...     ...        ...       ...      ...
    return df

new_col = tif_extraccion(archivos, new_df)
porcentaje = new_col['theta_r_15-30cm.tif'].isna().sum()
print(f'Cantidad de datos: {len(new_col)}\nTiene un {porcentaje/len(new_col)}% de NaN')
new_col.to_csv('tif_data')

Error index 74628 is out of bounds for axis 1 with size 9891


In [98]:
new_col.to_csv('test_dataframe')

In [94]:
porcentaje = new_col['theta_r_15-30cm.tif'].isna().sum()
print(f'Cantidad de datos: {len(new_col)}\nTiene un {porcentaje/len(new_col)}% de NaN')

Cantidad de datos: 1247
Tiene un 0.4578989574979952% de NaN


In [49]:
new_df = pd.concat([df.Longitud, df.Latitud], axis = 1)


,Longitud,Latitud
0,-73.7855,-42.4686
1,-73.7855,-42.4686
2,-72.6983,-42.4087
3,-73.7889,-40.7083
4,-73.6047,-40.5750
...,...,...
1242,-72.9781,-36.5932
1243,-73.1032,-36.7543
1244,-73.1032,-36.7543
1245,-73.1032,-36.7543
